## Intro
This notebook demonstrates the usage of the `MC_sep.py`, which is a simple tool running MC for optimization of the alignment column separation into two subsets with minimal entropy.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from functools import partial

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

from MC_sep import *

np.random.seed(42)

In [3]:
! mkdir -p ./test_samples/

## Extract a test column
`label_column.py` allows to extract any column from the alignment and label it based on `header_flag` argument. It will also create random initial labeling to start the MC from.

In [4]:
! python label_column.py --help

Usage: label_column.py [OPTIONS]

  Command-line tool selects a columns from the `input_alignment` and
  constructs a binary labels of characters in the column

Options:
  -i, --input_alignment TEXT   a path to the input alignment  [required]
  -H, --header_flag TEXT       if a sequence header contains this flag, it
                               will be assigned class 1, and 0 otherwise
                               [required]

  -R, --random_bin_sep         if flag is provided, random initial separation
                               into binary classes is added

  -n, --column_number INTEGER  the number of column in the alignment starting
                               from 0; if not provided a random column is
                               selected

  -o, --output_path FILENAME   path to write an output; if not provided,
                               stdout is used

  -s, --skip INTEGER           if provided, this number of sequences from the
                               start

Here we extract a `n-th` column from the alignment, create a "true" binary labels based on the presence of "out" in the sequence header, and create a random initial labeling. Note also that `-s 1` allows to skip the first (annotation) sequence.

In [5]:
! python label_column.py -i ./test_samples/repr_aln_corrected.fasta -H out -R -s 1 -n 87 -o ./test_samples/col.txt

In the output, the first line is an extracted column, the second line is the true labeling (1 for "out" containing sequences), and the third is a random initial labeling.

In [6]:
cat ./test_samples/col.txt

RKRMRRRRNRERRRRRTSARRGGSSSDDDSSSSQRAEKRRNNNEMERKETEKREQKRRRRSTRRRRRRRRRMMNKKNNMCKNSTENRWEERKQQQKKRTQMNQSSRRRKTTNRRSEKKRCQSRINNISTQELQRRRSNQRRRRSSGRTTS
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 0 1 1 0 1 0 0 1 0 1 1 1 0 1 0 0 1 0 1 1 1 1 1 0 1 0 1 1 0 0 1 1 1 0 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 1 0 0 1 1 0 1 1 0 0 0 1 1 1 0 1 1 0 1 0 1 0 1 0 1 1 1 1 1 0 1 0 1 0 1 0 0 1 1 0 1 1 1 0 1 1 0 1 1 0 1 1 0 0 1 1 1 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 1 1


## Encode the column

`encode_input` takes this file and encodes it into numpy arrays. True labels are preserved, and sequence characters are encoded into numbers for a faster entropy calculation. However, passing `init=False` allows to disregard the initial labeling and create a new one.

In [7]:
with open('./test_samples/col.txt') as f:
    col, true_lab, init_lab, classes = encode_input(f, init_present=False)
print(*true_lab, sep='')
print(*init_lab, sep='')

000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011111111111111111111111111111111111111111111
010001000100001011101011111111001110100000111110110101011000000001101111010111010101001011111111111001111111101011010110101001101110000000001011100001


## Run the simulation
First we run the simulation using `run` function which gradually optimizes the initial separation into the one which divides the column into subsets with minimal entropy.

In [8]:
?run

Signature:
run(
    column: numpy.ndarray,
    labels: numpy.ndarray,
    true_labels: numpy.ndarray,
    steps: Sequence[Tuple[float, Callable[[numpy.ndarray], numpy.ndarray]]],
    n_steps: int,
    classes: Sequence[int],
    temp: float,
    min_members: Union[int, NoneType] = None,
    output_freq: Union[int, NoneType] = None,
    output_buffer: Union[IO, NoneType] = None,
)
Docstring:
Runs a simple MCMC optimizing the distribution of labels.

:param column: Array of encoded column characters
:param labels: Array of initial binary labels
:param true_labels: Original labels needed to score labels
:param steps: A sequence of pairs (`prob`, `step`) where `step` is a step function
(taking an array of labels and outputing an array of labels),
and a `prob` is a probability to choose this step.
It is assumed that probabilities correctly sum to 1.
:param n_steps: A number of steps to run
:param classes: A sequence of available classes (types of labels)
:param min_members: A minimum number

At each step, the proposal is generated based on the current state. 
Currently, the `step` function is based on flipping the labels (0 to 1 and vice versa) at $M * len(sequence)$ random positions.The acceptance probability is given as $e^{-T * (gain(proposal) - gain(current))}$ where $T$ is a (unitless) temperature parameter, and the $gain(x)=entropy(column) - entropy(set_1) - entropy(set_2)$. We seek to find a sequence with the highest gain in entropy after we merge two subsequences back into a column. The hightest gain is achieved when the entropy of subsets is the lowest.

First we need to construct a move set. 
Each move is a `step` function taking a single argument (an array of labels) and returning a somehow changed array. 
Each step has associated probability for it to occur. Step probabilites must sum to 1.0.

In [9]:
steps = [
    (0.5, partial(mutate_label, mut_pool=classes)),
    (0.5, flip_pair)]

In [10]:
results = run(
    col, labels=init_lab, true_labels=true_lab, steps=steps, 
    n_steps=1000000, classes=classes, min_members=30, output_freq=100000, temp=50)

>100000|0.0715|0.496
0 1 0 1 0 1 0 0 0 1 1 1 0 0 1 0 0 0 0 0 1 0 0 1 1 0 1 1 1 1 0 1 1 0 0 0 1 1 0 0 0 0 1 0 0 1 1 0 1 1 0 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 0 1 0 0 1 1 0 1 0 0 1 1 0 0 1 0 1 0 1 0 1 0 1 1 0 1 1 0 1 0 1 0 1 1 1 1 1 1
>200000|0.1652|0.5282
1 0 1 1 1 0 1 1 1 1 1 1 0 0 1 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 0 1 0 0 1 1 0 1 1 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 0 1 1 1 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 0 0 1 1 1 0 0 1 1 0 1 0 0 1 0 1 1 1 0 1 1 0 0 0 0 1 0 0 1 1 0 0 1 1 1 1 1 0 0 1
>300000|0.1825|0.5505
1 1 1 0 0 1 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 1 1 1 1 1 0 0 0 1 1 1 0 0 0 1 1 1 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1 0 0 0 0 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 0 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 0 1 1 0 1 1 1 1 1 1 1
>400000|0.1557|0.5068
1 1 1 0 1 1 1

`results` contains the info of a best solution found. 
- The first element is a step number. 
- The second is an associated dE score. 
- The third is a column score (based on true labels as well)
- The last element is an array of labels

It can be fasta-like formatted using `fmt_output` function

In [11]:
print(results)
print(fmt_output(*results))

(372601, 0.36933253980067393, 0.5491334800054173, array([1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1]))
>372601|0.3693|0.5491
1 0 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 1 1 1 0 0 1 1 1 1 1 1 0 0 0 0 0 1 1 1 0 1 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 1 1 1 0 1 0 1 1 0 0 0 1 0 0 0 1 0 1 0 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 0 0 0 1 1 1 0 0 1 0 1 1 1 0 1 0 1 1 1 0 1 0 1 1 0 0 0 1 0 1 1 1 1 1 1 1 0 1 0 0 1


Let's compare the initial labeling to the best one found

In [12]:
print(f"initial entropy gain: {de_score(init_lab, col, classes=classes)}")
print(f"final entropy gain: {de_score(results[-1], col, classes=classes)}")

initial entropy gain: -3.000165529166529
final entropy gain: -2.3883051680760823


Finally, let's do the same via command-line interface. The output of the program will be the same as calling `fmt_results(*best_solution)`.

In [13]:
! python3 MC_sep.py --help

Usage: MC_sep.py [OPTIONS]

  The tool runs MC for optimization of the alignment column separation into
  two subsets with minimal entropy.

Options:
  -i, --inp FILENAME              a path to a file where the first line is a
                                  sequence of characters, the second line is a
                                  true labeling, and the third line is
                                  optional initial (guess) labeling
                                  [required]

  -I, --use_init                  a flag whether the third line -- guess
                                  labeling -- is to be used

  -N, --n_steps INTEGER           a number of steps to run the algorithm
  -c, --classes TEXT              a sequence of comma-separated possible
                                  classes; if not provided, it will be
                                  inferred from true labels

  -C, --min_members TEXT          a minimum number of members each class
                        

Only one argument is required -- an input file -- the rest have default values.

The tool outputs the best solution found during the run.

In [14]:
! python3 MC_sep.py -i ./test_samples/col.txt

>7442|0.5119|0.5871
1 1 1 1 1 1 1 1 0 0 0 1 1 1 0 1 1 1 1 0 0 1 0 1 1 1 0 1 0 1 1 1 1 0 1 1 0 1 1 1 0 0 0 0 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 0 0 1 1 1 0 1 1 0 0 1 0 0 1 1 1 0 0 0 1 1 1 1 0 1 0 0 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 0 1 0 0 1 1 1 0 0 1 0 1 1 1 1 0 0 1 0 1 1 1 1 0 0 1 1 1


To access intermediate states, `-O` argument should be provided. If provided together with `-o` with a filename, the progress will be printed there.

In [15]:
! python3 MC_sep.py -i ./test_samples/col.txt -O 10000 -o ./test_samples/col.log.txt -N 100000 > /dev/null

In [16]:
! head ./test_samples/col.log.txt

>10000|0.5388|0.6502
0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 1 0 1 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 0 0 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 1 1 1 0 1 0 0 1 1 0 1 1 0 1 0 0 1 1 0 1 0 0 0 0 1 1 1 0 0 1 1
>20000|0.5695|0.619
1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0 0 0 1 0 1 0 1 0 1 1 0 1 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 1 0 1 1 0 0 0 1 0 1 1 1 1 1 1 0 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 0 1 1 1 0 1 0 1 1 1 0 0 1 0 1 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1
>30000|0.4729|0.5739
0 1 0 1 0 0 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 1 1 0 0 1 0 1 1 0 0 1 0 0 0 1 0 1 1 1 0 0 1 0 0 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0 0 0 1 1 1 1 0 0 1 0 1 1 0 0 0 1 0 1 1 0 0 1 1 1 1 0 1 0 1 0 0 1 1 0 1 0 0 1 0 0 0 1 0 1 1 0 0 1 0 0 1 1 0 0 1 0 1
>40000|0.3732|0.5648
1 1 0 1 1 0 1 1 0

## Performance
Finally, let's check the time performance of the most utilized functions

In [17]:
%%time
run(col, labels=init_lab, true_labels=true_lab, steps=steps, 
    n_steps=1000000, classes=classes, min_members=30, output_freq=None, temp=50);

CPU times: user 51.3 s, sys: 989 ms, total: 52.3 s
Wall time: 51.7 s


(60124,
 0.37925537133774956,
 0.6020509259754533,
 array([0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
        1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1,
        0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
        1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1]))

In [18]:
col_score = partial(column_score, labels_true=true_lab, column=col, weights=(np.log2(20), 1), classes=(1, 0))
%timeit col_score(init_lab)

11.6 µs ± 7.57 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
de_score_numba_ = partial(de_score_numba, column=col, classes=classes, min_members=None, e_column=None)
%timeit de_score_numba_(col)

The slowest run took 5.11 times longer than the fastest. This could mean that an intermediate result is being cached.
8.8 µs ± 7.54 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%timeit entropy(col)

1.5 µs ± 9.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
